# Compare results

## Setup

In [1]:
import pandas as pd
import numpy as np
import pickle 
from functools import reduce
import utils
import os
import time 

# set the directory containing results files to analyze
RESULTS_DIR = "results" 
# directory where dataframes with summarized results etc will be stored
SUMMARY_OUTPUT_DIR = "summaries"

## Load results of classification

In [2]:
results_lst = []
filepaths_lst = []
for filename in os.listdir(RESULTS_DIR):
    path = f"{RESULTS_DIR}/{filename}"
    with open(path, "rb") as f:
        filepaths_lst.append(path)
        print(f"Reading data from {path}")
        results_lst.append(pickle.load(f))

Reading data from results/dtw_results_for_ARIMA_statio_1658748438_00.pkl
Reading data from results/dtw_results_for_ARIMA_statio_1658747687_00.pkl
Reading data from results/dtw_results_for_ARIMA_statio_1658748399_00.pkl
Reading data from results/dtw_results_for_HMM_1658746205_00.pkl
Reading data from results/hmm_results_for_HMM_1658746351_00.pkl
Reading data from results/dtw_results_for_ARIMA_statio_1658748426_00.pkl
Reading data from results/dtw_results_for_HMM_1658746186_00.pkl
Reading data from results/hmm_results_for_HMM_1658746261_00.pkl
Reading data from results/dtw_results_for_HMM_1658746296_00.pkl
Reading data from results/dtw_results_for_HMM_1658746464_00.pkl
Reading data from results/hmm_results_for_HMM_1658746296_00.pkl
Reading data from results/dtw_results_for_HMM_1658746351_00.pkl
Reading data from results/dtw_results_for_ARIMA_statio_1658748332_00.pkl
Reading data from results/hmm_results_for_HMM_1658746205_00.pkl
Reading data from results/hmm_results_for_HMM_1658746464_00

In [4]:
for i, r in enumerate(results_lst):
    with open(r["data_filename"], "rb") as f:
        data = pickle.load(f)
        print(f"Info about generated data in {i}:")
        print(data.keys())

Info about generated data in 0:
dict_keys(['generating_model', 'data_filename', 'time_index', 'models_lst', 'labels_df', 'all_X_samples', 'indices_splits_lst', 'metadata', 'description'])
Info about generated data in 1:
dict_keys(['generating_model', 'data_filename', 'time_index', 'models_lst', 'labels_df', 'all_X_samples', 'indices_splits_lst', 'metadata', 'description'])
Info about generated data in 2:
dict_keys(['generating_model', 'data_filename', 'time_index', 'models_lst', 'labels_df', 'all_X_samples', 'indices_splits_lst', 'metadata', 'description'])
Info about generated data in 3:
dict_keys(['generating_model', 'data_filename', 'time_index', 'models_lst', 'labels_df', 'all_X_samples', 'all_Z_samples', 'indices_splits_lst', 'metadata', 'description'])
Info about generated data in 4:
dict_keys(['generating_model', 'data_filename', 'time_index', 'models_lst', 'labels_df', 'all_X_samples', 'all_Z_samples', 'indices_splits_lst', 'metadata', 'description'])
Info about generated data 

## Compare accuracies

- `time_id`: time index of file with generated data / results
- `gen_with`: type of model the data was generated with ("HMM" / "ARIMA" / "ARIMA_all_statio" etc)
- `n_train`: number of train samples per model
- `n_test`: number of test samples per model
- `min_len`: min sample size parameter used in generation
- `max_len`: max sample size parameter used in generation
- `cls_with`: type of model used to classify samples ("HMM" / "DTW")
- `variant`: variant of classification, e.g. for HMM: "AIC" or "BIC", for DTW: "1NN" or "5NN" derived as a key from `predictions_dfs` list
- `acc`: accuracy of predictions


In [5]:
time_id = []
gen_with = []
n_train = []
n_test = []
min_len = []
max_len = []
cls_with = []
variants = []
accs = []

for r in results_lst:
    for variant, acc in r["accuracies"].items():
        with open(r["data_filename"], "rb") as f:
            data = pickle.load(f)
            time_id.append(r["time_index"])
            gen_with.append(data["generating_model"])
            n_train.append(data["metadata"]["N_TRAIN_SAMPLES_PER_MODEL"])
            n_test.append(data["metadata"]["N_TEST_SAMPLES_PER_MODEL"])
            min_len.append(data["metadata"]["MIN_SAMPLE_LEN"])
            max_len.append(data["metadata"]["MAX_SAMPLE_LEN"])
            cls_with.append(r["classificator"])
            variants.append(variant)
            accs.append(acc)

acc_summary = pd.DataFrame({
    "time_id": time_id,
    "gen_with": gen_with,
    "n_train": n_train,
    "n_test": n_test,
    "min_len": min_len,
    "max_len": max_len,
    "cls_with": cls_with,
    "variant": variants,
    "acc": accs
})

display(acc_summary.sort_values(by=['acc'], ascending = False))
            

,time_id,gen_with,n_train,n_test,min_len,max_len,cls_with,variant,acc
29,1658746464,HMM,30,100,100,1000,HMM,BIC,1.000000
28,1658746464,HMM,30,100,100,1000,HMM,AIC,1.000000
8,1658746351,HMM,5,100,100,1000,HMM,AIC,1.000000
9,1658746351,HMM,5,100,100,1000,HMM,BIC,1.000000
15,1658746261,HMM,5,100,50,150,HMM,BIC,0.998889
14,1658746261,HMM,5,100,50,150,HMM,AIC,0.998889
21,1658746296,HMM,30,100,50,150,HMM,BIC,0.994444
20,1658746296,HMM,30,100,50,150,HMM,AIC,0.994444
27,1658746205,HMM,30,100,20,50,HMM,BIC,0.980000
26,1658746205,HMM,30,100,20,50,HMM,AIC,0.980000


In [6]:
t = int(time.time())
summary_data = {
    "creation_date": t,
    "results_files": filepaths_lst,
    "accuracies_df": acc_summary

}
with open(f"{SUMMARY_OUTPUT_DIR}/summary_{t}.pkl", "wb") as f:
    pickle.dump(summary_data, f)